In [ ]:
import numpy as np
import MDAnalysis as mda

# Load the membrane simulation trajectory and topology
universe = mda.Universe('membrane.psf', 'membrane.dcd')

# Select the lipid atoms for curvature calculation
lipids = universe.select_atoms('resname LIP')

# Calculate the local curvature at each lipid
curvatures = []
for lipid in lipids:
    neighbors = lipid.neighbors
    if len(neighbors) < 2:
        continue
    tangents = [neighbor.position - lipid.position for neighbor in neighbors]
    angles = []
    for i in range(len(tangents)-1):
        for j in range(i+1, len(tangents)):
            angle = np.arccos(np.dot(tangents[i], tangents[j]) /
                              (np.linalg.norm(tangents[i]) * np.linalg.norm(tangents[j])))
            angles.append(angle)
    if len(angles) < 2:
        curvature = np.nan
    else:
        curvature = np.sum(angles) / (len(angles) * np.linalg.norm(np.mean(tangents, axis=0)))
    curvatures.append(curvature)

# Calculate the mean curvature at each lipid
mean_curvatures = []
for lipid in lipids:
    neighbors = lipid.neighbors
    if len(neighbors) < 2:
        continue
    tangents = [neighbor.position - lipid.position for neighbor in neighbors]
    curvatures = []
    for i in range(len(tangents)-1):
        for j in range(i+1, len(tangents)):
            curvature = np.arccos(np.dot(tangents[i], tangents[j]) /
                                  (np.linalg.norm(tangents[i]) * np.linalg.norm(tangents[j])))
            curvatures.append(curvature)
    if len(curvatures) < 2:
        mean_curvature = np.nan
    else:
        principal_curvatures = np.linalg.eigvals(np.outer(tangents, tangents.T))
        mean_curvature = 0.5 * np.mean(principal_curvatures)
    mean_curvatures.append(mean_curvature)

# Calculate the Gaussian curvature at each lipid
gaussian_curvatures = []
for lipid, mean_curvature in zip(lipids, mean_curvatures):
    neighbors = lipid.neighbors
    if len(neighbors) < 2:
        continue
    tangents = [neighbor.position - lipid.position for neighbor in neighbors]
    curvatures = []
    for i in range(len(tangents)-1):
        for j in range(i+1, len(tangents)):
            curvature = np.arccos(np.dot(tangents[i], tangents[j]) /
                                  (np.linalg.norm(tangents[i]) * np.linalg.norm(tangents[j])))
            curvatures.append(curvature)
    if len(curvatures) < 2:
        gaussian_curvature = np.nan
    else:
        gaussian_curvature = (2 * np.pi - np.sum(curvatures)) / (mean_curvature ** 2 * len(neighbors))
    gaussian_curvatures.append(gaussian_curvature)

# Fit the Gaussian curvature data to a quadratic polynomial
coefficients = np.polyfit(np.arange(len(gaussian_curvatures)), gaussian_curvatures, 2)

# Calculate the Gaussian curvature modulus
gaussian_curvature_modulus = 1 / (coefficients[2] * 2)

# Print the Gaussian curvature modulus
